In [1]:
import os

import numpy as np
import tensorflow as tf
import joblib
import pandas as pd

Datos que se usan:
- annual_inc (float)
- emp_length (categorica)
- home_ownership (categorica)
- purpose (categorica)
- zip_code (int)
- open_acc (int)
- dti (float): esta variable hay que calcularla

Para calcular el DTI (Debt-to-Income Ratio) a partir de la información que le proporcionas al usuario, puedes seguir estos pasos:

1. **Ingresos Mensuales:**  
   Si el usuario te da su ingreso anual, divídelo entre 12 para obtener el ingreso mensual. Por ejemplo, si declara que gana \$36,000 al año, su ingreso mensual es:

$$
   \text{Ingreso Mensual} = \frac{36000}{12} = 3000 \, \text{USD}
$$

2. **Pagos Mensuales de Deuda:**  
   Pide al usuario que sume todos los pagos mensuales de sus deudas (como préstamos, tarjetas de crédito, hipotecas, etc.). Por ejemplo, si el total de sus pagos mensuales es de \$600, ese es el numerador.

3. **Fórmula del DTI:**  
   El DTI se calcula dividiendo el total de pagos mensuales de deuda entre el ingreso mensual y, opcionalmente, multiplicando por 100 para expresarlo en porcentaje:

$$
   \text{DTI} = \frac{\text{Pagos Mensuales de Deuda}}{\text{Ingreso Mensual}} \times 100
$$

   Siguiendo el ejemplo:

$$
   \text{DTI} = \frac{600}{3000} \times 100 = 20\%
$$

Este 20% indica que el 20% del ingreso mensual del usuario se destina al pago de sus deudas. Es un indicador útil para evaluar la capacidad de pago del solicitante.

In [ ]:
# Cargar modelo y preprocesador
model = tf.keras.models.load_model(os.path.join('..', 'models', 'loan_model_2.h5'))
preprocessor = joblib.load(os.path.join('..', 'models', 'preprocessor.pkl'))

In [ ]:
def predict_with_optimal_threshold(model, X, threshold=0.326):
    """
    Realiza predicciones usando el threshold optimizado
    """
    y_pred_proba = model.predict(X)
    return (y_pred_proba >= threshold).astype(int)

In [ ]:
def probability_to_score_stretched(prob, base_score=300, max_score=850, threshold=0.326, 
									expansion_factor_low=3, expansion_factor_high=0.7):
    """
    Convierte probabilidades en puntajes de crédito con expansión no lineal 
    para distribuir mejor en los extremos.
    
    Args:
        prob (float): Probabilidad de default.
        base_score (int): Puntaje base.
        max_score (int): Puntaje máximo.
        threshold (float): Valor de corte óptimo.
        expansion_factor_low (float): Factor para expandir la parte baja del rango.
        expansion_factor_high (float): Factor para expandir la parte alta del rango.
        
    Returns:
        score (float): Puntaje de crédito ajustado.
    """
    # Invertir la probabilidad para que mayor valor sea mejor score
    inverted_prob = 1 - prob
    
    # Punto de corte invertido
    inverted_threshold = 1 - threshold
    
    # Determinar si es un score alto o bajo
    if inverted_prob >= inverted_threshold:  # Buenos clientes
        # Normalizar la probabilidad en el rango de buenos
        normalized = (inverted_prob - inverted_threshold) / (1 - inverted_threshold)
        # Aplicar expansión no lineal
        transformed = normalized ** expansion_factor_high
        # Mapear al rango superior
        mid_score = 550  # Punto medio del rango
        score = mid_score + (max_score - mid_score) * transformed
    else:  # Malos clientes
        # Normalizar la probabilidad en el rango de malos
        normalized = inverted_prob / inverted_threshold
        # Aplicar expansión no lineal para los scores bajos
        transformed = normalized ** expansion_factor_low
        # Mapear al rango inferior
        mid_score = 550  # Punto medio del rango
        score = base_score + (mid_score - base_score) * transformed
    
    # Asegurar que el score esté dentro del rango permitido
    score = np.clip(score, base_score, max_score)
    
    return score

In [ ]:
data = pd.DataFrame({'prob': np.linspace(0, 1, 100)})
data_processed = preprocessor.transform(data)	
predictions_proba = model.predict(data_processed)
predictions = predict_with_optimal_threshold(model, data_processed)
credit_score = probability_to_score_stretched(predictions_proba[0])